# 指定パターン数を出力する

In [1]:
import random
import json
import lzma

GREGORIAN_START = 1582 # グレゴリオ暦の開始年
GREGORIAN_END = 9999 # これは出題上の上限

OLYMPIC_START = 1896 # 近代オリンピック開催年

COLORS_COUNT_OF_SAMPLES = 999

In [28]:
def get_random_years(not_list=[]):
    
    # 2000年を平均として、標準偏差50年で開始年を取得
    while True:
        year_start = max(min(int(random.gauss(2000,50)),GREGORIAN_END),GREGORIAN_START)
        year_end = year_start + max(min(int(random.gauss(100,100)),200),1)
        year_tuple = (year_start, year_end)
        year_tuple_str = " ".join(list(map(str,year_tuple)))
        if year_tuple_str not in not_list:
            return year_tuple
    
    raise ValueError('無限ループしたようです')
    
get_random_years()

(2031, 2115)

In [29]:
def olympic_count(year_start, year_end):
    count = 0
    for year in range(year_start, year_end+1):
        if year<1896: # 近代オリンピック開催前
            pass
        elif year%4 == 0 and year not in [1916,1940,1944,2020]: # 4の倍数の場合 and 戦争中とコロナによる延期は除く
            count += 1
        elif year in [2021]: # 4の倍数以外で唯一開催している年(コロナによる延期)
            count += 1
    
    return count

olympic_count(1582, 2023)

29

In [30]:
def output_json_xz(xz_filename, json_data):
    # JSONデータを整形して文字列として取得
    # ensure_ascii=False すると日本語がエスケープされなくなるためjsonの可読性はあがりますが互換性は低くなります。注意。
    json_str = json.dumps(json_data, indent=4, sort_keys=True, ensure_ascii=False)

    # JSONファイルをXZ形式で圧縮する
    with lzma.open(xz_filename, 'wb') as xz_out:
        xz_out.write(json_str.encode('utf-8'))

In [31]:
def output_all(out_xz_filename, pattern):
    pattern_keta = len(str(pattern)) # 桁数を数える（文字列に変換後、文字列長で取得できる）
    output_index = 0
    all_inputs = []
    json_data = {}

    # ランダムで入れる
    for answer_id in range(1, pattern+1):
        input_list = []
        output_list = []

        # randomで入力文字列を取得する
        input_years = get_random_years(all_inputs)
        input_str = " ".join(list(map(str,input_years)))
        input_list.append(input_str)
        all_inputs.append(input_str)

        # オリンピックカウントの解答を求める
        output_list.append(str(olympic_count(input_years[0],input_years[1])))

        # 入力と出力をJSONに詰める
        answer_key = f"answer{answer_id:0{pattern_keta}}"
        json_data[answer_key] = [input_list, output_list]

    # jsonファイルをxz出力する
    output_json_xz(out_xz_filename, json_data)

    # 重複がなければpatternと一致するはず
    print('all_inputs count=',len(set(all_inputs)))

In [32]:
output_all("data/answer.json.xz", COLORS_COUNT_OF_SAMPLES)

all_inputs count= 999
